# Find train type correspondance

# Type of trains

In [167]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


#### Imports:

In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6485,application_1589299642358_0974,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
routes = spark.read.format('orc').load('/data/sbb/timetables/orc/routes')
stop_times = spark.read.format('orc').load('/data/sbb/timetables/orc/stop_times/000000_0')
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%local
import pandas as pd
mean_std_df = pd.read_pickle('pickle_delay_distribution')

In [5]:
%%send_to_spark -i mean_std_df -t df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'mean_std_df' as 'mean_std_df' to Spark kernel

In [6]:
routes.select(col('route_desc')).distinct().show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|   route_desc|
+-------------+
|          TGV|
|     Eurocity|
|Standseilbahn|
|  Regionalzug|
| RegioExpress|
|       S-Bahn|
| Luftseilbahn|
|   Sesselbahn|
|         Taxi|
|        Fähre|
|         Tram|
|          ICE|
|          Bus|
|   Gondelbahn|
|    Nacht-Zug|
|  Auoreisezug|
|     Eurostar|
|       Schiff|
|   Schnellzug|
|    Intercity|
|   InterRegio|
|     Extrazug|
|        Metro|
+-------------+

In [8]:
translate_route_desc = {
    'TGV': 'TGV',
    'Eurocity': 'EC',
    'tandseilbahn': 'AT',
    'Regionalzug': 'R',
    'RegioExpress': 'RE',
    'S-Bahn': 'S',
    'Luftseilbahn': '',
    'Sesselbahn': '',
    'Taxi': '',
    'Fähre': '',
    'Tram': 'Tram',
    'ICE': 'ICE',
    'Bus': 'Bus',
    'Gondelbahn': '',
    'Nacht-Zug': '',
    'Standseilbahn': 'AT',
    'Auoreisezug': 'ARZ',
    'Eurostar': 'EC',
    'Schiff': '',
    'Schnellzug': 'TGV',
    'Intercity': 'IC',
    'InterRegio': 'IR',
    'Extrazug': 'EXT',
    'Metro': 'Metro'
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Check T

In [43]:
actual.where(col('verkehrsmittel_text') == 'T').where(col('haltestellen_name') == 'Zürich, Bachmattstrasse').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:3849:113276-32002-1'), Row(fahrt_bezeichner=u'85:3849:134333-32002-1')]

In [53]:
actual.where(col('verkehrsmittel_text') == 'T').where(col('fahrt_bezeichner') == '85:3849:113276-32002-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+----------------+-------------------+
|haltestellen_name             |ankunftszeit    |verkehrsmittel_text|
+------------------------------+----------------+-------------------+
|Schlieren, Zentrum/Bahnhof    |28.12.2019 00:48|T                  |
|Schlieren, Wagonsfabrik       |28.12.2019 00:49|T                  |
|Schlieren, Gasometerbrücke    |28.12.2019 00:50|T                  |
|Schlieren, Mülligen           |28.12.2019 00:51|T                  |
|Zürich, Micafil               |28.12.2019 00:52|T                  |
|Zürich, Farbhof               |28.12.2019 00:53|T                  |
|Zürich, Bachmattstrasse       |28.12.2019 00:54|T                  |
|Zürich, Lindenplatz           |28.12.2019 00:55|T                  |
|Zürich, Grimselstrasse        |28.12.2019 00:56|T                  |
|Zürich, Kappeli               |28.12.2019 00:58|T                  |
|Zürich, Freihofstrasse        |28.12.2019 00:59|T                  |
|Zürich, Letzigrund 

#### From sbb.ch -> T is Tram

## Check B

In [37]:
actual.where(col('verkehrsmittel_text') == 'B').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:827:280700-02132-1'), Row(fahrt_bezeichner=u'85:827:280700-02132-1')]

In [52]:
actual.where(col('verkehrsmittel_text') == 'B').where(col('fahrt_bezeichner') == '85:827:280700-02132-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------------+-------------------+
|haltestellen_name    |ankunftszeit    |verkehrsmittel_text|
+---------------------+----------------+-------------------+
|Bern Bahnhof         |25.10.2019 05:15|B                  |
|Bundesplatz          |25.10.2019 05:17|B                  |
|Zytglogge            |25.10.2019 05:19|B                  |
|Viktoriaplatz        |25.10.2019 05:20|B                  |
|Salem                |25.10.2019 05:21|B                  |
|Schönburg            |25.10.2019 05:22|B                  |
|Rosengarten          |25.10.2019 05:23|B                  |
|Galgenfeld           |25.10.2019 05:24|B                  |
|Waldeck              |25.10.2019 05:26|B                  |
|Schosshaldenfriedhof |25.10.2019 05:26|B                  |
|Ostermundigen Bahnhof|25.10.2019 05:27|B                  |
|Zollgasse            |25.10.2019 05:29|B                  |
|Wegmühlegässli       |25.10.2019 05:30|B                  |
|Sportplatzweg        |2

#### From sbb.ch -> B is Bus

## Check NFB

In [47]:
actual.where(col('verkehrsmittel_text') == 'NFB').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:870:51001'), Row(fahrt_bezeichner=u'85:870:51001')]

In [51]:
actual.where(col('verkehrsmittel_text') == 'NFB').where(col('fahrt_bezeichner') == '85:870:51001')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+----------------+-------------------+
|haltestellen_name             |ankunftszeit    |verkehrsmittel_text|
+------------------------------+----------------+-------------------+
|Wangenried, Linde             |23.10.2018 05:30|NFB                |
|Wangenried, Unterdorf         |23.10.2018 05:31|NFB                |
|Wangen a.d. Aare, Unterführung|23.10.2018 05:32|NFB                |
|Wangen a.A., Bahnhof          |23.10.2018 05:34|NFB                |
|Wangen a.d.Aare, Zytglogge    |23.10.2018 05:35|NFB                |
|Herzogenbuchsee, Jurablick    |23.10.2019 05:21|NFB                |
|Wanzwil, Post                 |23.10.2019 05:22|NFB                |
|Röthenbach, Chrump            |23.10.2019 05:23|NFB                |
|Röthenbach b. H., Post        |23.10.2019 05:24|NFB                |
|Röthenbach, Schützenhaus      |23.10.2019 05:25|NFB                |
|Wangenried, Sägerei           |23.10.2019 05:27|NFB                |
|Wangenried, Schule 

#### From sbb.ch -> NFB is Bus, NF actually means 'NiederFlurzüge' meaning the bus can be used by disabled

## Check R

In [66]:
actual.where(col('verkehrsmittel_text') == 'R').select(col('fahrt_bezeichner')).limit(20).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'81:817000:5702:002'), Row(fahrt_bezeichner=u'81:817000:5703:001'), Row(fahrt_bezeichner=u'81:817000:5704:001'), Row(fahrt_bezeichner=u'81:817000:5706:001'), Row(fahrt_bezeichner=u'81:817000:5707:001'), Row(fahrt_bezeichner=u'81:817000:5708:001'), Row(fahrt_bezeichner=u'81:817000:5710:001'), Row(fahrt_bezeichner=u'81:817000:5715:001'), Row(fahrt_bezeichner=u'81:817000:5721:001'), Row(fahrt_bezeichner=u'81:817000:5722:001'), Row(fahrt_bezeichner=u'81:817000:5723:001'), Row(fahrt_bezeichner=u'81:817000:5725:001'), Row(fahrt_bezeichner=u'81:817000:5727:001'), Row(fahrt_bezeichner=u'81:817000:5729:001'), Row(fahrt_bezeichner=u'83:3270__:28318:000'), Row(fahrt_bezeichner=u'83:3270__:28318:000'), Row(fahrt_bezeichner=u'83:3270__:28318:000'), Row(fahrt_bezeichner=u'83:3270__:28318:000'), Row(fahrt_bezeichner=u'83:3270__:28330:000'), Row(fahrt_bezeichner=u'83:3270__:28330:000')]

In [69]:
actual.where(col('verkehrsmittel_text') == 'R').where(col('fahrt_bezeichner') == '83:3270__:28318:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------------+-------------------+
|haltestellen_name|ankunftszeit    |verkehrsmittel_text|
+-----------------+----------------+-------------------+
|Stabio           |01.09.2018 08:22|R                  |
|Mendrisio        |01.09.2018 08:27|R                  |
|Balerna          |01.09.2018 08:36|R                  |
|Chiasso          |01.09.2018 08:40|R                  |
|Stabio           |02.09.2018 08:22|R                  |
|Mendrisio        |02.09.2018 08:27|R                  |
|Balerna          |02.09.2018 08:36|R                  |
|Chiasso          |02.09.2018 08:40|R                  |
|Stabio           |03.09.2018 08:22|R                  |
|Mendrisio        |03.09.2018 08:27|R                  |
|Balerna          |03.09.2018 08:36|R                  |
|Chiasso          |03.09.2018 08:40|R                  |
|Stabio           |04.09.2018 08:22|R                  |
|Mendrisio        |04.09.2018 08:27|R                  |
|Balerna          |04.09.2018 0

#### From sbb.ch -> R is regional train

## Check M

In [71]:
actual.where(col('verkehrsmittel_text') == 'M').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:151:TL070-4506262507106362'), Row(fahrt_bezeichner=u'85:151:TL070-4506262507106362')]

In [72]:
actual.where(col('verkehrsmittel_text') == 'M').where(col('fahrt_bezeichner') == '85:151:TL070-4506262507106362')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+----------------+-------------------+
|haltestellen_name             |ankunftszeit    |verkehrsmittel_text|
+------------------------------+----------------+-------------------+
|Lausanne, Vigie               |23.10.2019 05:31|M                  |
|Lausanne, Montelly            |23.10.2019 05:33|M                  |
|Lausanne, Provence            |23.10.2019 05:34|M                  |
|Lausanne, Malley              |23.10.2019 05:35|M                  |
|Lausanne, Bourdonnette        |23.10.2019 05:38|M                  |
|Chavannes-R., UNIL-Chamberonne|23.10.2019 05:39|M                  |
|Chavannes-R., UNIL-Mouline    |23.10.2019 05:40|M                  |
|Ecublens VD, UNIL-Sorge       |23.10.2019 05:41|M                  |
|Ecublens VD, EPFL             |23.10.2019 05:43|M                  |
|Ecublens VD, Bassenges        |23.10.2019 05:44|M                  |
|Ecublens VD, Cerisaie         |23.10.2019 05:45|M                  |
|Chavannes-R., Croch

#### From sbb.ch -> M is Metro

## Check BN

In [74]:
actual.where(col('verkehrsmittel_text') == 'BN').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:773:9232-01353-1'), Row(fahrt_bezeichner=u'85:773:9232-01353-1')]

In [80]:
actual.where(col('verkehrsmittel_text') == 'BN').where(col('fahrt_bezeichner') == '85:773:9232-01353-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.09.2019 07:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+----------------+-------------------+
|haltestellen_name            |ankunftszeit    |verkehrsmittel_text|
+-----------------------------+----------------+-------------------+
|Watt, Geerenweg              |23.11.2019 01:27|BN                 |
|Watt, Schulhaus              |23.11.2019 01:28|BN                 |
|Watt                         |23.11.2019 01:28|BN                 |
|Adlikon b. R., Pächterried   |23.11.2019 01:29|BN                 |
|Adlikon bei Regensdorf       |23.11.2019 01:30|BN                 |
|Adlikon b. R., Sonnhalde     |23.11.2019 01:32|BN                 |
|Adlikon b. R., Sonnhalde West|23.11.2019 01:32|BN                 |
|Buchs ZH, Petermoos          |23.11.2019 01:34|BN                 |
|Buchs ZH, Linde              |23.11.2019 01:35|BN                 |
|Buchs ZH, Post               |23.11.2019 01:36|BN                 |
|Buchs-Dällikon, Bahnhof      |23.11.2019 01:37|BN                 |
|Buchs ZH, Industrie          |23.

#### From sbb.ch -> These are night weekend buses, so we don't want to count them. We only look at reasonable hours

## Check TN

In [78]:
actual.where(col('verkehrsmittel_text') == 'TN').select(col('fahrt_bezeichner')).limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:37:13059-00607-1'), Row(fahrt_bezeichner=u'85:37:13059-00607-1')]

In [79]:
actual.where(col('verkehrsmittel_text') == 'TN').where(col('fahrt_bezeichner') == '85:37:13059-00607-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+-------------------+
|haltestellen_name |ankunftszeit    |verkehrsmittel_text|
+------------------+----------------+-------------------+
|Heuwaage          |28.12.2019 01:31|TN                 |
|Zoo               |28.12.2019 01:33|TN                 |
|Dorenbach         |28.12.2019 01:34|TN                 |
|Binningen Oberdorf|28.12.2019 01:35|TN                 |
|Binningen Schloss |28.12.2019 01:36|TN                 |
|Bottmingermühle   |28.12.2019 01:38|TN                 |
|Batteriestrasse   |28.12.2019 01:39|TN                 |
|Bottmingen Schloss|28.12.2019 01:40|TN                 |
|Stallen           |28.12.2019 01:42|TN                 |
|Oberwil Zentrum   |28.12.2019 01:43|TN                 |
|Hüslimatt         |28.12.2019 01:45|TN                 |
|Therwil Zentrum   |28.12.2019 01:47|TN                 |
|Känelmatt         |28.12.2019 01:48|TN                 |
|Ettingen Bahnhof  |28.12.2019 01:51|TN                 |
+-------------

#### From sbb.ch -> These are night weekend trams, so we don't want to count them. We only look at reasonable hours

## Check KB

In [84]:
actual.where(col('verkehrsmittel_text') == 'KB').select(col('fahrt_bezeichner')).limit(20).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51009'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51013'), Row(fahrt_bezeichner=u'85:870:51018'), Row(fahrt_bezeichner=u'85:870:51018'), Row(fahrt_bezeichner=u'85:870:51018'), Row(fahrt_bezeichner=u'85:870:51018'), Row(fahrt_bezeichner=u'85:870:51018')]

In [85]:
actual.where(col('verkehrsmittel_text') == 'KB').where(col('fahrt_bezeichner') == '85:870:51013')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+----------------+-------------------+
|haltestellen_name             |ankunftszeit    |verkehrsmittel_text|
+------------------------------+----------------+-------------------+
|Inkwil, Dorf                  |24.01.2018 06:23|KB                 |
|Wangenried, Sägerei           |24.01.2018 06:25|KB                 |
|Wangenried, Schule            |24.01.2018 06:26|KB                 |
|Wangenried, Linde             |24.01.2018 06:27|KB                 |
|Wangenried, Unterdorf         |24.01.2018 06:28|KB                 |
|Wangen a.d. Aare, Unterführung|24.01.2018 06:29|KB                 |
|Wangen a.A., Bahnhof          |24.01.2018 06:31|KB                 |
|Wangen a.d.Aare, Zytglogge    |24.01.2018 06:32|KB                 |
|Inkwil, Dorf                  |24.01.2019 06:23|KB                 |
|Wangenried, Sägerei           |24.01.2019 06:25|KB                 |
|Wangenried, Schule            |24.01.2019 06:26|KB                 |
|Wangenried, Linde  

#### From sbb.ch -> Another type of bus

## Check SN

In [87]:
actual.where(col('verkehrsmittel_text') == 'SN').select(col('fahrt_bezeichner')).limit(20).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:907:000'), Row(fahrt_bezeichner=u'85:46:914:000'), Row(fahrt_bezeichner=u'85:46:914:000')]

In [88]:
actual.where(col('verkehrsmittel_text') == 'SN').where(col('fahrt_bezeichner') == '85:46:907:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------------+-------------------+
|haltestellen_name    |ankunftszeit    |verkehrsmittel_text|
+---------------------+----------------+-------------------+
|Zürich Kreuzplatz    |23.11.2019 01:06|SN                 |
|Zürich Hegibachplatz |23.11.2019 01:08|SN                 |
|Zürich, Wetlistrasse |23.11.2019 01:10|SN                 |
|Zürich Balgrist      |23.11.2019 01:12|SN                 |
|Zürich, Rehalp       |23.11.2019 01:14|SN                 |
|Waldburg             |23.11.2019 01:16|SN                 |
|Spital Zollikerberg  |23.11.2019 01:17|SN                 |
|Zollikerberg         |23.11.2019 01:18|SN                 |
|Waltikon             |23.11.2019 01:20|SN                 |
|Zumikon              |23.11.2019 01:21|SN                 |
|Maiacher             |23.11.2019 01:22|SN                 |
|Neue Forch           |23.11.2019 01:23|SN                 |
|Forch                |23.11.2019 01:25|SN                 |
|Scheuren             |2

#### From sbb.ch -> Night S, so we don't count them

## Check VAE

In [94]:
actual.where(col('verkehrsmittel_text') == 'VAE').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:82:2570:001'), Row(fahrt_bezeichner=u'85:82:72564:002')]

In [95]:
actual.where(col('verkehrsmittel_text') == 'VAE').where(col('fahrt_bezeichner') == '85:82:72564:002')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------------+-------------------+
|haltestellen_name|ankunftszeit    |verkehrsmittel_text|
+-----------------+----------------+-------------------+
|Herisau          |26.05.2018 07:12|VAE                |
|Wattwil          |26.05.2018 07:32|VAE                |
|Herisau          |27.05.2018 07:12|VAE                |
|Wattwil          |27.05.2018 07:32|VAE                |
+-----------------+----------------+-------------------+

#### From sbb.ch -> VAE is Voralpen-Express so trains in the Alps for tourism. We don't need to count them since we consider stations around Zürich

## Check BAT

In [97]:
actual.where(col('verkehrsmittel_text') == 'BAT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:185:827:000'), Row(fahrt_bezeichner=u'85:183:60:000')]

In [99]:
actual.where(col('verkehrsmittel_text') == 'BAT').where(col('fahrt_bezeichner') == '85:183:60:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '23.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+----------------+-------------------+
|haltestellen_name            |ankunftszeit    |verkehrsmittel_text|
+-----------------------------+----------------+-------------------+
|Giessbach See                |24.07.2019 10:51|BAT                |
|Oberried am Brienzersee (See)|24.07.2019 11:06|BAT                |
|Iseltwald (See)              |24.07.2019 11:16|BAT                |
|Bönigen                      |24.07.2019 11:35|BAT                |
|Interlaken Ost (See)         |24.07.2019 11:53|BAT                |
|Giessbach See                |24.08.2018 10:51|BAT                |
|Iseltwald (See)              |24.08.2018 11:09|BAT                |
|Niederried (See)             |24.08.2018 11:18|BAT                |
|Ringgenberg (See)            |24.08.2018 11:27|BAT                |
|Bönigen                      |24.08.2018 11:35|BAT                |
|Interlaken Ost (See)         |24.08.2018 11:53|BAT                |
|Giessbach See                |24.

#### From sbb.ch -> Some type of bus

## Check D

#### From sbb.ch -> D is Direct, like RegioExpress

## Check TER

In [113]:
actual.where(col('verkehrsmittel_text') == 'TER').select(col('fahrt_bezeichner')).distinct().limit(20).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:11:96558:004'), Row(fahrt_bezeichner=u'85:11:96211:001'), Row(fahrt_bezeichner=u'85:11:96133:002'), Row(fahrt_bezeichner=u'85:11:23437:001'), Row(fahrt_bezeichner=u'85:11:96235:002'), Row(fahrt_bezeichner=u'85:11:37983:002'), Row(fahrt_bezeichner=u'85:11:18156:003'), Row(fahrt_bezeichner=u'85:11:96433:001'), Row(fahrt_bezeichner=u'85:11:18110:001'), Row(fahrt_bezeichner=u'85:11:96670:002'), Row(fahrt_bezeichner=u'85:11:96413:001'), Row(fahrt_bezeichner=u'85:11:96243:002'), Row(fahrt_bezeichner=u'87:87____:96502:000'), Row(fahrt_bezeichner=u'85:11:96121:001'), Row(fahrt_bezeichner=u'85:11:96578:003'), Row(fahrt_bezeichner=u'85:11:96143:003'), Row(fahrt_bezeichner=u'85:11:96112:001'), Row(fahrt_bezeichner=u'85:11:96145:001'), Row(fahrt_bezeichner=u'85:11:96152:001'), Row(fahrt_bezeichner=u'85:11:96501:001')]

In [116]:
actual.where(col('verkehrsmittel_text') == 'TER')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------+-------------------+
|haltestellen_name|ankunftszeit|verkehrsmittel_text|
+-----------------+------------+-------------------+
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|La Chaux-de-Fonds|            |TER                |
|Basel SBB        |            |TER                |
|La Chaux-de-Fonds|            |TER                |
|Basel SBB        |            |TER                |
|La Chaux-de-Fonds|            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER                |
|Basel SBB        |            |TER           

#### From sbb.ch -> TER is a French railway, so we remove it since we look at Zürich

## Check EXT

In [127]:
actual.where(col('verkehrsmittel_text') == 'EXT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:86:23460:001'), Row(fahrt_bezeichner=u'85:86:23589:002')]

In [131]:
actual.where(col('verkehrsmittel_text') == 'EXT')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .where(col('ankunftszeit') >= '20.11.2019 00:00').sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------------+-------------------+
|haltestellen_name|ankunftszeit    |verkehrsmittel_text|
+-----------------+----------------+-------------------+
|Untervaz-Trimmis |20.11.2019 00:00|EXT                |
|Haldenstein      |20.11.2019 00:06|EXT                |
|Chur Wiesental   |20.11.2019 00:08|EXT                |
|Chur             |20.11.2019 00:10|EXT                |
|Chur             |20.11.2019 00:11|EXT                |
|Felsberg         |20.11.2019 00:22|EXT                |
|Chur West        |20.11.2019 00:25|EXT                |
|Chur             |20.11.2019 00:28|EXT                |
|Palézieux        |20.11.2019 07:37|EXT                |
|Romont           |20.11.2019 07:49|EXT                |
|Fribourg/Freiburg|20.11.2019 08:05|EXT                |
|Bern             |20.11.2019 08:28|EXT                |
|Genève           |20.11.2019 10:20|EXT                |
|Chur West        |20.11.2019 22:01|EXT                |
|Felsberg         |20.11.2019 2

#### This looks very much like ExtraZug

## Check RB

#### From sbb.ch -> RB is RegionalBahn

## Check BT

In [153]:
actual.where(col('verkehrsmittel_text') == 'BT').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:823:687118-09072-1'), Row(fahrt_bezeichner=u'85:823:689210-09073-1')]

In [160]:
actual.where(col('verkehrsmittel_text') == 'BT').where(col('fahrt_bezeichner') == '85:823:689210-09073-1')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----------------+-------------------+
|haltestellen_name      |ankunftszeit    |verkehrsmittel_text|
+-----------------------+----------------+-------------------+
|Riehen Grenze          |                |BT                 |
|Riehen Grenze          |                |BT                 |
|Lörracherstrasse       |01.12.2019 02:02|BT                 |
|Weilstrasse            |01.12.2019 02:03|BT                 |
|Fondation Beyeler      |01.12.2019 02:04|BT                 |
|Riehen Dorf            |01.12.2019 02:05|BT                 |
|Bettingerstrasse       |01.12.2019 02:06|BT                 |
|Pfaffenloh             |01.12.2019 02:07|BT                 |
|Burgstrasse            |01.12.2019 02:08|BT                 |
|Niederholzboden        |01.12.2019 02:09|BT                 |
|Habermatten            |01.12.2019 02:10|BT                 |
|Eglisee                |01.12.2019 02:12|BT                 |
|Hirzbrunnen/Claraspital|01.12.2019 02:14|BT           

#### From sbb.ch -> BT is a night tram, so we ignore it

## Check BAV

In [162]:
actual.where(col('verkehrsmittel_text') == 'BAV').select(col('fahrt_bezeichner')).distinct().limit(2).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(fahrt_bezeichner=u'85:185:18:000'), Row(fahrt_bezeichner=u'85:185:24:000')]

In [163]:
actual.where(col('verkehrsmittel_text') == 'BAV').where(col('fahrt_bezeichner') == '85:185:18:000')\
        .select(col('haltestellen_name'), col('ankunftszeit'), col('verkehrsmittel_text'))\
        .sort('ankunftszeit').show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+-------------------+
|haltestellen_name |ankunftszeit    |verkehrsmittel_text|
+------------------+----------------+-------------------+
|Flüelen (See)     |                |BAV                |
|Flüelen (See)     |                |BAV                |
|Flüelen (See)     |                |BAV                |
|Flüelen (See)     |                |BAV                |
|Sisikon (See)     |06.10.2019 12:24|BAV                |
|Brunnen (See)     |06.10.2019 12:47|BAV                |
|Treib             |06.10.2019 12:57|BAV                |
|Gersau (See)      |06.10.2019 13:14|BAV                |
|Beckenried (See)  |06.10.2019 13:31|BAV                |
|Vitznau           |06.10.2019 13:48|BAV                |
|Weggis            |06.10.2019 14:05|BAV                |
|Hertenstein (See) |06.10.2019 14:14|BAV                |
|Verkehrshaus-Lido |06.10.2019 14:35|BAV                |
|Luzern Bahnhofquai|06.10.2019 14:47|BAV                |
|Sisikon (See)

#### From sbb.ch -> BT is a night tram, so we ignore it

# Replace value

With the above manual checks, [this pdf](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=2ahUKEwiB_sLqlL3pAhVKXhoKHSUUD28QFjAAegQIARAB&url=https%3A%2F%2Fcompany.sbb.ch%2Fcontent%2Fdam%2Finternet%2Fcorporate%2Ffr%2Fsbb-als-geschaeftspartner%2Fflotte-unterhalt%2Fonestopshop%2FLeistungskatalog-2020.pdf.sbbdownload.pdf&usg=AOvVaw1faXmNvW-PypBiC-Hz7c_W) and online research. We can map `verkehrsmittel_text` to correct values to map it to the `route_desc` in `routes.txt`

In [164]:
replace_actual = {
    'BUS': 'Bus', # Buses
    'B': 'Bus',
    'NFB': 'Bus',
    'KB': 'Bus',
    'BAT': 'Bus',
    'Trm': 'Tram', # Trams
    'T': 'Tram',
    'TRAM': 'Tram',
    'ATZ': 'ARZ', #AutoZug
    'D': 'RE', # Regional
    'RB': 'R',
    'M': 'Metro', # Metro
    'ICE': 'IC', # InterCityExpress, but routes.txt doesn't have that category
    'IRE': 'IR', # InterRegioExpress, but routes.txt doesn't have that category
    'BN': '', # Night
    'TN': '',
    'SN': '',
    'BT': '',
    'VAE': '', # Panorama trains in the Alps
    'PE': '',
    'TER': '', # France
    'TE2': '',
    'RJX': '', # International
    'null': '' # Other
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
@udf("string")
def replace_verkehrsmittel_text(text):
    if text in replace_actual.keys():
        return replace_actual[text]
    else:
        return text

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
counts.select(replace_verkehrsmittel_text(col('verkehrsmittel_text')).alias('verkehrsmittel_text'), col('count'))\
        .groupBy('verkehrsmittel_text').sum()\
        .sort('sum(count)', ascending=False).show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+
|verkehrsmittel_text|sum(count)|
+-------------------+----------+
|                Bus| 338771078|
|                   | 322747219|
|                  S|  66583644|
|               Tram|  66513733|
|                  R|  25871287|
|                 RE|   6363409|
|                 IR|   4036535|
|                 IC|   2300761|
|              Metro|   1010960|
|                 EC|    221206|
|                EXT|     75905|
|                TGV|     44461|
|                ZUG|     31525|
|                 NB|     26011|
|                EXB|     17817|
|                GEX|     16724|
|                 NJ|     15180|
|                BEX|     14766|
|                 RJ|     14702|
|                  P|     11240|
|                 CC|      7469|
|                 AG|       982|
|                BAV|       724|
|                 NZ|       613|
|                 AT|       527|
|                ARZ|       274|
|                 EN|       109|
|         

In [58]:
counts = actual.groupBy('verkehrsmittel_text').count()
counts.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[verkehrsmittel_text: string, count: bigint]